Master Thesis Machine Learing  
Gwen Hirsch  
2022

# Train ACD and Baselines on Power System and Weather Time-Series

#### imports and variable definitions

In [1]:
#own imports
import pandas as pd
import seaborn as sns
import numpy as np
import torch
import networkx as nx
import datetime
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib import colors
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import pytz
# from scipy import stats
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
import gc
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

#acd imports
from __future__ import division
from __future__ import print_function
from collections import defaultdict
import time
import argparse
import os
import math
import itertools
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as tdist
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score
import torch.optim as optim
from torch.optim import lr_scheduler
from abc import abstractmethod

C:\ProgramData\Anaconda3\envs\base2\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# lists that contain subsets of names later needed for filtering the dataframe columns

# columns I want to predict
vars_of_interest = ['load_actual', 'solar_generation', 'wind_generation' , 'price_day_ahead']

# countries to include in dataset
prefixes = ['DE_', 'FR_', 'CH_', 'GB_']

# lists of the columns with variables of interest
cols_of_interest = [prefix+var for prefix in prefixes for var in vars_of_interest]
cols_of_interest_DE = ['DE_'+var for var in vars_of_interest]
cols_of_interest_FR = ['FR_'+var for var in vars_of_interest]
cols_of_interest_CH = ['CH_'+var for var in vars_of_interest]
cols_of_interest_GB = ['GB_'+var for var in vars_of_interest]

# lists of the columns with weather variables
vars_weather = ['temperature', 'radiation_direct_horizontal', 'radiation_diffuse_horizontal']
cols_weather = [prefix+var for prefix in prefixes for var in vars_weather]
cols_weather_DE = ['DE_'+var for var in vars_weather]
cols_weather_FR = ['FR_'+var for var in vars_weather]
cols_weather_CH = ['CH_'+var for var in vars_weather]
cols_weather_GB = ['GB_'+var for var in vars_weather]

# all 7 quantities to predict
vars_predict = vars_of_interest+vars_weather

# save time column names
cols_time = ['year', 'month', 'day', 'hour', 'weekday']

# save column names in lists and with different orders for plotting...
# ...according to countries
all_vars_of_interest = [prefix+var for prefix in prefixes for var in vars_of_interest+vars_weather]
all_vars_of_interest_DE = cols_of_interest_DE + cols_weather_DE
all_vars_of_interest_FR = cols_of_interest_FR + cols_weather_FR
all_vars_of_interest_CH = cols_of_interest_CH + cols_weather_CH
all_vars_of_interest_GB = cols_of_interest_GB + cols_weather_GB

# ...according to variables
all_vars_of_interest_varfirst = [prefix+var for var in vars_of_interest+vars_weather for prefix in prefixes]

# all original columns in final dataset
all_cols_to_keep     = ['utc_timestamp']+ all_vars_of_interest +cols_time
all_cols_to_keep_DE  = ['utc_timestamp']+ [var for var in all_vars_of_interest if var.startswith('DE_')] +cols_time
all_cols_to_keep_FR  = ['utc_timestamp']+ [var for var in all_vars_of_interest if var.startswith('FR_')] +cols_time
all_cols_to_keep_CH  = ['utc_timestamp']+ [var for var in all_vars_of_interest if var.startswith('CH_')] +cols_time
all_cols_to_keep_GB  = ['utc_timestamp']+ [var for var in all_vars_of_interest if var.startswith('GB_')] +cols_time

# used in dataset generation
# contains all aditional time features 
cols_ts_DE = all_vars_of_interest_DE
cols_ts_FR = all_vars_of_interest_FR
cols_ts_CH = all_vars_of_interest_CH
cols_ts_GB = all_vars_of_interest_GB
cols_ts_all = all_vars_of_interest
cols_ts_long = vars_of_interest+vars_weather
cols_to_cycle = ['month', 'day', 'hour', 'weekday']
cols_timedims_cycle = [timedim+version for timedim in cols_to_cycle for version in ['_sin', '_cos']]
cols_to_normalize_DE = cols_ts_DE+['year']
cols_to_normalize_all = cols_ts_all+['year']
cols_to_normalize_long = cols_ts_long+['year', 'ID']

idx_firstmonday = 96
idx_lastsunday = 43775

In [3]:
# labels for correlation matrix

# labels if correlation matrix is done for Germany only
labels_vars_full = {}
for v in all_vars_of_interest:
    if 'load_actual' in v:
        labels_vars_full[v] = (v.partition('_')[0]+' total load', '[MW]')
    elif 'load_forecast' in v:
        labels_vars_full[v] = (v.partition('_')[0]+' load forecast', '[MW]')
    elif 'solar_generation' in v:
        labels_vars_full[v] = (v.partition('_')[0]+' solar generation', '[MW]')
    elif 'wind_generation' in v:
        labels_vars_full[v] = (v.partition('_')[0]+' wind generation', '[MW]')
    elif 'price_day_ahead' in v:
        if v.partition('_')[0] != 'GB':
            labels_vars_full[v] = (v.partition('_')[0]+' price day-ahead', '[EUR]')
        else:
            labels_vars_full[v] = (v.partition('_')[0]+' price day-ahead', '[GBP]')
    elif 'temperature' in v:
        labels_vars_full[v] = (v.partition('_')[0]+' temperature', '[°C]')
    elif 'radiation_direct_horizontal' in v:
        labels_vars_full[v] = (v.partition('_')[0]+' radiation direct', '[W/m²]')
    elif 'radiation_diffuse_horizontal' in v:
        labels_vars_full[v] = (v.partition('_')[0]+' radiation diffuse', '[W/m²]')
        
# labels if correlation matrix is done for all countries
labels_vars_short = {}
for v in all_vars_of_interest:
    if 'load_actual' in v:
        labels_vars_short[v] = (v.partition('_')[0]+' LO', '[MW]')
    elif 'solar_generation' in v:
        labels_vars_short[v] = (v.partition('_')[0]+' SO', '[MW]')
    elif 'wind_generation' in v:
        labels_vars_short[v] = (v.partition('_')[0]+' WI', '[MW]')
    elif 'price_day_ahead' in v:
        if v.partition('_')[0] != 'GB':
            labels_vars_short[v] = (v.partition('_')[0]+' PR', '[EUR]')
        else:
            labels_vars_short[v] = (v.partition('_')[0]+' PR', '[GBP]')
    elif 'temperature' in v:
        labels_vars_short[v] = (v.partition('_')[0]+' TE', '[°C]')
    elif 'radiation_direct_horizontal' in v:
        labels_vars_short[v] = (v.partition('_')[0]+' R1', '[W/m²]')
    elif 'radiation_diffuse_horizontal' in v:
        labels_vars_short[v] = (v.partition('_')[0]+' R2', '[W/m²]')

#### settings for training runs

In [4]:
# choose dataset
suffix = "_energy_DE_1d"
suffix = "_energy_all_1d"
suffix = "_energy_long_1d"
# suffix = "_energy_DE_7d"
# suffix = "_energy_all_7d"
# suffix = "_energy_long_7d"

# choose acd version -> either a gcn encoder or variational distribution
version = 'gcn'
# version = 'variational'

# choose lstm_format=True if data set has to have lstm format (e.g. for rnn baseline) 
# NOTE: run_rnn has to be set to True for the rnn baseline to train
lstm_format = False 
# lstm_format = True

# whether to run ACD model
run_acd = False
# run_acd = True
if run_acd:
    if lstm_format:
        lstm_format = False

# whether to run MLR baseline
run_mlr = False
run_mlr = True
if run_mlr:
    if lstm_format:
        lstm_format = False

# whether to run LSTM baseline
run_rnn = False
# run_rnn = True
if run_rnn:
    if not lstm_format:
        lstm_format = True

# further settings
epochs = 100
batch_size = 16
no_cuda = False
# no_cuda=True

## Specify settings and load normed dataframes

.npy input files for acd have shape  
[num_samples, num_ts, num_timesteps, num_dims]   
e.g. [3*365, 7, 24/48, 1+num_features]  
(in acd code [num_sims, num_atoms, num_timesteps, num_dims]) 

#### load dataframe and norm params

In [5]:
if suffix == "_energy_DE_1d":
    num_samples_train = 3*365+1 
    num_samples_valid = 365
    num_samples_test = 365
    cols_ts = cols_ts_DE
    cols_timedims = ['year']+cols_timedims_cycle
    num_ts = len(cols_ts)
    num_timesteps = 24
    num_dims = 1 +len(cols_timedims)
    if run_rnn or lstm_format:
        lstm_format = True
        suffix = suffix[:-3]+'_lstm'
        num_samples_train = num_samples_train*num_timesteps
        num_samples_valid = num_samples_valid*num_timesteps
        num_samples_test = num_samples_test*num_timesteps
        num_dims = num_ts + len(cols_timedims)
        num_timesteps=1
    dataframe = pd.read_csv('data/normed' +suffix+'.csv', parse_dates=['utc_timestamp'])
    df_norm_params = pd.read_csv('data/norm_params' +suffix+'.csv')
    
if suffix == "_energy_all_1d":
    num_samples_train = 3*365+1 
    num_samples_valid = 365
    num_samples_test = 365
    cols_ts = cols_ts_all
    cols_timedims = ['year']+cols_timedims_cycle
    num_ts = len(cols_ts)
    num_timesteps = 24
    num_dims = 1 +len(cols_timedims)
    if run_rnn or lstm_format:
        lstm_format = True
        suffix = suffix[:-3]+'_lstm'
        num_samples_train = num_samples_train*num_timesteps
        num_samples_valid = num_samples_valid*num_timesteps
        num_samples_test = num_samples_test*num_timesteps
        num_dims = num_ts + len(cols_timedims)
        num_timesteps=1
    dataframe = pd.read_csv('data/normed' +suffix+'.csv', parse_dates=['utc_timestamp'])
    df_norm_params = pd.read_csv('data/norm_params' +suffix+'.csv')
        
if suffix == "_energy_long_1d":
    num_samples_train = (3*365+1)*len(prefixes)
    num_samples_valid = 365*len(prefixes)
    num_samples_test = 365*len(prefixes)
    cols_ts = cols_ts_long
    cols_timedims = ['year']+cols_timedims_cycle+['ID']
    num_ts = len(cols_ts)
    num_timesteps = 24
    num_dims = 1 +len(cols_timedims)
    if run_rnn or lstm_format:
        lstm_format=True 
        suffix = suffix[:-3]+'_lstm'
        num_samples_train = num_samples_train*num_timesteps
        num_samples_valid = num_samples_valid*num_timesteps
        num_samples_test = num_samples_test*num_timesteps
        num_dims = num_ts + len(cols_timedims)
        num_timesteps=1
    dataframe = pd.read_csv('data/normed' +suffix+'.csv', parse_dates=['utc_timestamp'])
    df_norm_params = pd.read_csv('data/norm_params' +suffix+'.csv')
        
if suffix == "_energy_DE_7d":
    num_samples_train = 3*52
    num_samples_valid = 52
    num_samples_test = 52
    cols_ts = cols_ts_DE
    cols_timedims = ['year']+cols_timedims_cycle
    num_ts = len(cols_ts)
    num_timesteps = 24*7
    num_dims = 1 +len(cols_timedims)
    if run_rnn or lstm_format:
        lstm_format=True 
        suffix = suffix[:-3]+'_lstm'
        num_samples_train = num_samples_train*num_timesteps
        num_samples_valid = num_samples_valid*num_timesteps
        num_samples_test = num_samples_test*num_timesteps
        num_dims = num_ts + len(cols_timedims)
        num_timesteps=1
    idx_start = idx_firstmonday#df['weekday'][df['weekday'] == 0].index[0]
    idx_end = idx_firstmonday+260*7*24    
    dataframe = pd.read_csv('data/normed' +suffix+'.csv', parse_dates=['utc_timestamp'])
    df_norm_params = pd.read_csv('data/norm_params' +suffix+'.csv')
        
if suffix == "_energy_all_7d":
    num_samples_train = 3*52
    num_samples_valid = 52
    num_samples_test  = 52
    cols_ts = cols_ts_all
    cols_timedims = ['year']+cols_timedims_cycle
    num_ts = len(cols_ts)
    num_timesteps = 24*7
    num_dims = 1 + len(cols_timedims)
    if run_rnn or lstm_format:
        lstm_format=True 
        suffix = suffix[:-3]+'_lstm'
        num_samples_train = num_samples_train*num_timesteps
        num_samples_valid = num_samples_valid*num_timesteps
        num_samples_test = num_samples_test*num_timesteps
        num_dims = num_ts + len(cols_timedims)
        num_timesteps=1
    idx_start = idx_firstmonday#df['weekday'][df['weekday'] == 0].index[0]
    idx_end = idx_firstmonday+260*7*24    
    dataframe = pd.read_csv('data/normed' +suffix+'.csv', parse_dates=['utc_timestamp'])
    df_norm_params = pd.read_csv('data/norm_params' +suffix+'.csv')
        
if suffix == "_energy_long_7d":
    num_samples_train = 3*52*len(prefixes)
    num_samples_valid = 52*len(prefixes)
    num_samples_test = 52*len(prefixes)
    cols_ts = cols_ts_long
    cols_timedims = ['year']+cols_timedims_cycle+['ID']
    num_ts = len(cols_ts)
    num_timesteps = 24*7
    num_dims = 1 +len(cols_timedims)
    if run_rnn or lstm_format:
        lstm_format=True 
        suffix = suffix[:-3]+'_lstm'
        num_samples_train = num_samples_train*num_timesteps
        num_samples_valid = num_samples_valid*num_timesteps
        num_samples_test = num_samples_test*num_timesteps
        num_dims = num_ts + len(cols_timedims)
        num_timesteps=1
    idx_start = idx_firstmonday*len(prefixes)
    idx_end = idx_firstmonday*len(prefixes)+260*7*24*len(prefixes)
    dataframe = pd.read_csv('data/normed' +suffix+'.csv', parse_dates=['utc_timestamp'])
    df_norm_params = pd.read_csv('data/norm_params' +suffix+'.csv')
        

In case some shape error comes up, comment in the cell below.

In [6]:
# feat_train_energyx = np.load('data/feat_train' +suffix+'.npy')
# feat_valid_energyx = np.load('data/feat_valid' +suffix+'.npy')
# feat_test_energyx = np.load('data/feat_test' +suffix+'.npy')
# feat_predict_energyx = np.load('data/feat_predict' +suffix+'.npy')

# edges_train_energyx    = np.load('data/edges_train' +suffix+'.npy')
# edges_valid_energyx    = np.load('data/edges_valid' +suffix+'.npy')
# edges_test_energyx     = np.load('data/edges_test' +suffix+'.npy')
# edges_predict_energyx  = np.load('data/edges_predict' +suffix+'.npy')
# norm_params_energyx    = pd.read_csv('data/norm_params' +suffix+ '.csv')

# print(feat_train_energyx.shape)
# print(feat_valid_energyx.shape)
# print(feat_test_energyx.shape)
# print(feat_predict_energyx.shape)
# if run_rnn:
#     target_train_energyx = np.load('data/target_train' +suffix+'.npy')
#     target_valid_energyx = np.load('data/target_valid' +suffix+'.npy')
#     target_test_energyx  = np.load('data/target_test' +suffix+'.npy')
#     print(target_train_energyx.shape)
#     print(target_valid_energyx.shape)
#     print(target_test_energyx.shape)
# print(edges_train_energyx.shape)
# print(edges_valid_energyx.shape)
# print(edges_test_energyx.shape)
# print(edges_predict_energyx.shape)
# display(norm_params_energyx)

## Model implementations

### -------------------- ACD Code --------------------

This acd implementation is adapted from the original implementation at  
https://github.com/loeweX/AmortizedCausalDiscovery.  
All individual files from there are stored in ACD_code.py

Optionally, instead of importing the .py file, one can copy the cells from the    
ACD_code_notebook.ipynb in here for easier handling when editing.

In [7]:
from ACD_code import *

#### train.py

In [8]:
# from __future__ import division
# from __future__ import print_function

# from collections import defaultdict

# import time
# import numpy as np
# import torch

# from model.modules import *
# from utils import arg_parser, logger, data_loader, forward_pass_and_eval
# from model import utils, model_loader


def train():
    best_val_loss = np.inf
    best_epoch = 0

    for epoch in range(args.epochs):
        t_epoch = time.time()
        train_losses = defaultdict(list)

        for batch_idx, minibatch in enumerate(train_loader):

            data, relations, temperatures = unpack_batches(args, minibatch)

            optimizer.zero_grad()

            losses, _, _, _ = forward_pass_and_eval(
                args,
                encoder,
                decoder,
                data,
                relations,
                rel_rec,
                rel_send,
                args.hard,
                edge_probs=edge_probs,
                log_prior=log_prior,
                temperatures=temperatures,
            )

            loss = losses["loss"]

            loss.backward()
            optimizer.step()

            train_losses = append_losses(train_losses, losses)

        string = logs.result_string("train", epoch, train_losses, t=t_epoch)
        logs.write_to_log_file(string)
        logs.append_train_loss(train_losses)
        scheduler.step()

        if args.validate:
            val_losses = val(epoch)
            val_loss = np.mean(val_losses["loss"])
            if val_loss < best_val_loss:
                print("Best model so far, saving...")
                logs.create_log(
                    args,
                    encoder=encoder,
                    decoder=decoder,
                    optimizer=optimizer,
                    accuracy=np.mean(val_losses["acc"]),
                )
                best_val_loss = val_loss
                best_epoch = epoch
        elif (epoch + 1) % 100 == 0:
            logs.create_log(
                args,
                encoder=encoder,
                decoder=decoder,
                optimizer=optimizer,
                accuracy=np.mean(train_losses["acc"]),
            )

        logs.draw_loss_curves()

    return best_epoch, epoch


def val(epoch):
    t_val = time.time()
    val_losses = defaultdict(list)

    if args.use_encoder:
        encoder.eval()
    decoder.eval()

    for batch_idx, minibatch in enumerate(valid_loader):

        data, relations, temperatures = unpack_batches(args, minibatch)

        with torch.no_grad():
            losses, _, _, _ = forward_pass_and_eval(
                args,
                encoder,
                decoder,
                data,
                relations,
                rel_rec,
                rel_send,
                True,
                edge_probs=edge_probs,
                log_prior=log_prior,
                testing=True,
                temperatures=temperatures,
            )

        val_losses = append_losses(val_losses, losses)

    string = logs.result_string("validate", epoch, val_losses, t=t_val)
    logs.write_to_log_file(string)
    logs.append_val_loss(val_losses)

    if args.use_encoder:
        encoder.train()
    decoder.train()

    return val_losses


def test(encoder, decoder, epoch):
    args.shuffle_unobserved = False
    # args.prediction_steps = 49
    test_losses = defaultdict(list)

    if args.load_folder == "":
        ## load model that had the best validation performance during training
        if args.use_encoder:
            encoder.load_state_dict(torch.load(args.encoder_file))
        decoder.load_state_dict(torch.load(args.decoder_file))

    if args.use_encoder:
        encoder.eval()
    decoder.eval()

    for batch_idx, minibatch in enumerate(test_loader):

        data, relations, temperatures = unpack_batches(args, minibatch)
        # print(data.shape)
        # print(relations.shape)
        # print(data.size(2))
        # print(args.timesteps)

        with torch.no_grad():
            assert (data.size(2) - args.timesteps) >= args.timesteps

            data_encoder = data[:, :, : args.timesteps, :].contiguous()
            data_decoder = data[:, :, args.timesteps : -1, :].contiguous()

            losses, _, _, _, = forward_pass_and_eval(
                args,
                encoder,
                decoder,
                data,
                relations,
                rel_rec,
                rel_send,
                True,
                data_encoder=data_encoder,
                data_decoder=data_decoder,
                edge_probs=edge_probs,
                log_prior=log_prior,
                testing=True,
                temperatures=temperatures,
            )

        test_losses = append_losses(test_losses, losses)

    string = logs.result_string("test", epoch, test_losses)
    logs.write_to_log_file(string)
    logs.append_test_loss(test_losses)

    logs.create_log(
        args,
        decoder=decoder,
        encoder=encoder,
        optimizer=optimizer,
        final_test=True,
        test_losses=test_losses,
    )




### -------------------- MLR Code --------------------

In [9]:
def run_MLR(df, cols_ts, cols_timedims, 
            num_samples_train=(3*365+1)*24,
            num_samples_valid=365*24, 
            num_samples_test=365*24,
            epochs=100,
            suffix='_energy',
            print_every=100,
            lr=0.1
            ):
    
    num_ts = len(cols_ts)
    len_df = len(df)
    offset_valid = num_samples_train
    offset_test = num_samples_train+num_samples_valid
    linear_regression_models = {}
    test_losses = {}
    for target in cols_ts:
        x_whole = torch.FloatTensor(df.loc[:(len_df-1)-1, cols_ts+cols_timedims].values).T
        y_whole = torch.FloatTensor(df.loc[1:(len_df-1), target].values).T
        
        x_train = torch.FloatTensor(df.loc[0:num_samples_train-1, cols_ts+cols_timedims].values.T)#.values)
        y_train = torch.FloatTensor(df.loc[1:num_samples_train, target].values.T)#.values)
        
        x_valid = torch.FloatTensor(df.loc[offset_valid+0:offset_valid+num_samples_valid-1, 
                                          cols_ts+cols_timedims].values).T
        y_valid = torch.FloatTensor(df.loc[offset_valid+1:offset_valid+num_samples_valid, 
                                          target].values).T        
        
        x_test = torch.FloatTensor(df.loc[offset_test+0:offset_test+num_samples_test-1-1, 
                                          cols_ts+cols_timedims].values).T
        y_test = torch.FloatTensor(df.loc[offset_test+1:offset_test+num_samples_test-1, 
                                          target].values).T
        
        
        n_predictors = x_train.shape[0]
        A = torch.randn((1, n_predictors), requires_grad=True)
        b = torch.randn(1, requires_grad=True)
        
        optimizer = optim.Adam([A, b], lr=lr)

        for i in range(epochs):
            
            optimizer.zero_grad()
            train_pred = A.mm(x_train) + b
            train_loss = ((train_pred - y_train)**2).sum()/num_samples_train
            train_loss.backward()
            optimizer.step()
            
            valid_pred = A.detach()@x_valid + b.detach()
            val_loss = ((valid_pred - y_valid)**2).sum()/num_samples_valid
            
            if (i+1)%print_every==0:
                string = f"i={i+1} target {target} train_loss {train_loss} val_loss {val_loss}" 
                logs.write_to_log_file(string)
                
                
        test_pred = A.detach()@x_test + b.detach()
        test_loss = ((test_pred - y_test)**2).sum()/num_samples_test
        test_losses[target]=test_loss.item()
        
        string = f"target = {target}, test loss = {test_loss}" 
        logs.write_to_log_file(string)   
        
        linear_regression_models[target]=(A, b)
        
        df[target+'_pred'] = 0
        df.loc[1:,target+'_pred'] = (A.detach()@x_whole + b.detach())[0].tolist()
        
    avg_test_loss = np.mean(list(test_losses.values()))
    string = f"average test loss= {avg_test_loss}" 
    logs.write_to_log_file(string)   
    
    return linear_regression_models, df, test_losses, avg_test_loss

def predict_MLR(df, MLR_models, cols_ts, cols_timedims):
    for target in cols_ts:
        x = torch.FloatTensor(df.loc[0:num_samples_train-1,
                                           cols_ts+cols_timedims].values).T
        y = torch.FloatTensor(df.loc[1:num_samples_train, 
                                           target].values).T
        (A, b) = MLR_models[target]

        df[target+'_pred'] = 0
        df.loc[1:,target+'_pred'] = (A.detach()@x + b.detach())[0].tolist()        
    return df

### -------------------- RNN Code --------------------

In [10]:
class RNN_baseline(nn.Module):
    def __init__(self, lstm_layers, input_dim, target_dim, batch_size, hidden_dim, device):
        super(RNN_baseline, self).__init__()
        self.lstm_layers = lstm_layers
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.target_dim = target_dim
        self.device = device
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.lstm_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_dim, self.target_dim)
        
    def forward(self, inputs):
        hidden_state = torch.zeros(self.lstm_layers, self.batch_size, self.hidden_dim).to(self.device)
        cell_state = torch.zeros(self.lstm_layers, self.batch_size, self.hidden_dim).to(self.device)
        hidden = (hidden_state, cell_state)
        
        outputs, hidden = self.lstm(inputs)
        
        return self.linear(outputs).squeeze()

In [11]:
def train_rnn(args):
    best_val_loss = np.inf
    best_epoch = 0

    for epoch in range(args.epochs):
        t_epoch = time.time()
        train_losses = defaultdict(list)
        
        (train_loader, 
         valid_loader,
         test_loader,
         loc_max,
         loc_min,
         vel_max,
         vel_min,
        ) = load_data(args)

        for batch_idx, minibatch in enumerate(train_loader):
            
            data, target, _ = unpack_batches(args, minibatch)

            optimizer.zero_grad()
            
            losses, output = forward_pass_rnn(
                args,
                rnn,
                data,
                target)

            loss = losses["loss_mse"]

            loss.backward()
            optimizer.step()
    
            train_losses = append_losses(train_losses, losses)

        string = logs.result_string("train", epoch, train_losses, t=t_epoch)
        logs.write_to_log_file(string)
        logs.append_train_loss(train_losses)
        scheduler.step()

        if args.validate:
            val_losses = val_rnn(epoch, rnn_args)
            val_loss = np.mean(val_losses["loss_mse"])
            if val_loss < best_val_loss:
                print("Best model so far, saving...")
                logs.create_log(
                    rnn_args,
                    rnn=rnn,
                    optimizer=optimizer,
                )
                best_val_loss = val_loss
                best_epoch = epoch
        elif (epoch + 1) % 100 == 0:
            logs.create_log(
                rnn_args,
                rnn=rnn,
                optimizer=optimizer,
            )

        logs.draw_loss_curves()

    return best_epoch, epoch


def val_rnn(epoch, args):
    t_val = time.time()
    val_losses = defaultdict(list)

    rnn.eval()

    for batch_idx, minibatch in enumerate(valid_loader):

        data, target, _ = unpack_batches(args, minibatch)

        with torch.no_grad():
            losses, output = forward_pass_rnn(
                args,
                rnn,
                data,
                target
            )

        val_losses = append_losses(val_losses, losses)

    string = logs.result_string("validate", epoch, val_losses, t=t_val)
    logs.write_to_log_file(string)
    logs.append_val_loss(val_losses)

    rnn.train()

    return val_losses


def test_rnn(rnn, epoch, args):
    test_losses = defaultdict(list)

    if args.load_folder == "":
        ## load model that had the best validation performance during training
        rnn.load_state_dict(torch.load(args.rnn_file))
    
    rnn.eval()    
    
    for batch_idx, minibatch in enumerate(test_loader):

        data, target, _ = unpack_batches(args, minibatch)

        with torch.no_grad():
            losses, output = forward_pass_rnn(
                args,
                rnn,
                data,
                target#.unsqueeze(1)
            )

        test_losses = append_losses(test_losses, losses)

    string = logs.result_string("test", epoch, test_losses)
    logs.write_to_log_file(string)
    logs.append_test_loss(test_losses)

    logs.create_log(   
        args,
        rnn=rnn,
        optimizer=optimizer,
        final_test=True,
        test_losses=test_losses
    )

## Run models

### -------------------- run ACD on energy data --------------------

#### train model

In [12]:
training_samples=num_samples_train
test_samples=num_samples_test
timesteps=num_timesteps
num_atoms=num_ts
dims=num_dims
if version == 'gcn':
    dont_use_encoder = False
elif version == 'variational':
    dont_use_encoder = True 

args = parse_args(
    epochs=epochs, 
    training_samples=training_samples, 
    test_samples=test_samples,
    shuffle_traindata=True,
    suffix=suffix,
    timesteps=timesteps,
    num_atoms=num_atoms,
    dims=dims,
    no_cuda=no_cuda,
    dont_use_encoder=dont_use_encoder,
    batch_size=batch_size
)

In [13]:
if run_acd:
    start_time = time.time()
    print(f"Starting_time: {datetime.datetime.fromtimestamp(start_time)}")
    
    logs = Logger(args)

    if args.GPU_to_use is not None:
        logs.write_to_log_file("Using GPU #" + str(args.GPU_to_use))

    (
        train_loader,
        valid_loader,
        test_loader,
        loc_max,
        loc_min,
        vel_max,
        vel_min,
    ) = load_data(args)

    rel_rec, rel_send = create_rel_rec_send(args, args.num_atoms)

    encoder, decoder, optimizer, scheduler, edge_probs = load_model(
        args, loc_max, loc_min, vel_max, vel_min
    )

    logs.write_to_log_file(encoder)
    logs.write_to_log_file(decoder)

    if args.prior != 1:
        assert 0 <= args.prior <= 1, "args.prior not in the right range"
        prior = np.array(
            [args.prior]
            + [
                (1 - args.prior) / (args.edge_types - 1)
                for _ in range(args.edge_types - 1)
            ]
        )
        logs.write_to_log_file("Using prior")
        logs.write_to_log_file(prior)
        log_prior = torch.FloatTensor(np.log(prior))
        log_prior = log_prior.unsqueeze(0).unsqueeze(0)

        if args.cuda:
            log_prior = log_prior.cuda()
    else:
        log_prior = None

    if args.global_temp:
        args.categorical_temperature_prior = get_categorical_temperature_prior(
            args.alpha, args.num_cats, to_cuda=args.cuda
        )

    ##Train model
    try:
        if args.test_time_adapt:
            raise KeyboardInterrupt

        best_epoch, epoch = train()

    except KeyboardInterrupt:
        best_epoch, epoch = -1, -1

    print("Optimization Finished!")
    logs.write_to_log_file("Best Epoch: {:04d}".format(best_epoch))

    if args.test:
        test(encoder, decoder, epoch)
    
    if args.dont_use_encoder:
        edge_probs_path = os.path.join(args.log_path, "edge_probs_final.pt")
        torch.save(edge_probs, edge_probs_path)
        args.edge_probs_file = edge_probs_path
    
    logs.write_to_log_file('Data: '+ suffix)
    logs.write_to_log_file('Version: '+ version)
    
    end_time = time.time()
    total_time = end_time-start_time
    print(f'Total Time: {time.strftime("%H:%M:%S", time.gmtime(total_time))}')
    logs.write_to_log_file(f'Total Time: {time.strftime("%H:%M:%S", time.gmtime(total_time))}')
    

### -------------------- run MLR on energy data --------------------

#### train mlr baseline

In [14]:
mlr_args = parse_args(
    epochs=epochs,
    training_samples=num_samples_train*num_timesteps, 
    test_samples=num_samples_test*num_timesteps,
    shuffle_traindata=True,
    suffix=suffix+'_MLR',
    timesteps=1,
    num_atoms=num_ts,
    dims=num_dims,
    no_cuda=no_cuda,
    dont_use_encoder=dont_use_encoder,
    batch_size=batch_size
)

In [15]:
if run_mlr:
    start_time = time.time()
    print(f"Starting_time: {datetime.datetime.fromtimestamp(start_time)}")
    
    logs = Logger(mlr_args)

    MLR_models, df_MLRpred_norm, test_losses, avg_test_loss = run_MLR(dataframe, 
                                  cols_ts, cols_timedims,
                                  num_samples_train=mlr_args.training_samples,
                                  num_samples_valid=num_samples_valid,
                                  num_samples_test=mlr_args.test_samples,
                                  epochs=epochs, print_every=10)
    
    np.save(os.path.join(mlr_args.log_path, "MLR_models"+suffix[:-3]+".npy"), MLR_models)
    
    # df_MLRpred = denormalize_columns(df_MLRpred_norm, suffix=suffix)
    # if save_pred_csv:
    #     df_MLRpred.to_csv('data/mlr_pred_'+ suffix + '.csv', index=False)
        
    logs.write_to_log_file('Data: '+ suffix)
    logs.write_to_log_file('Version: '+ version)
    
    end_time = time.time()
    total_time = end_time-start_time
    logs.write_to_log_file(f'Total Time: {time.strftime("%H:%M:%S", time.gmtime(total_time))}')

Starting_time: 2022-12-13 20:40:54.312457
<class 'str'>
Namespace(seed=969491451, GPU_to_use=None, epochs=100, batch_size=16, lr=0.0005, lr_decay=200, gamma=0.5, training_samples=105216, test_samples=35040, shuffle_traindata=True, prediction_steps=10, encoder_hidden=256, decoder_hidden=256, encoder='mlp', decoder='mlp', prior=1, edge_types=2, dont_use_encoder=False, lr_z=0.1, global_temp=False, load_temperatures=False, alpha=2, num_cats=3, unobserved=0, model_unobserved=0, dont_shuffle_unobserved=False, teacher_forcing=0, suffix='_energy_long_1d_MLR', timesteps=1, num_atoms=7, dims=11, datadir='./data', save_folder='logs', expername='', sym_save_folder='../logs', load_folder='', test_time_adapt=False, lr_logits=0.01, num_tta_steps=100, dont_skip_first=False, temp=0.5, hard=False, no_validate=False, no_cuda=False, var=5e-07, encoder_dropout=0.0, decoder_dropout=0.0, no_factor=False, f='C:\\Users\\GwenH\\AppData\\Roaming\\jupyter\\runtime\\kernel-4180d1bd-7365-43f6-9be5-09d102f5e672.json

In [16]:
dataframe

,utc_timestamp,week,year,month,day,hour,weekday,ID,load_actual,solar_generation,wind_generation,price_day_ahead,temperature,radiation_direct_horizontal,radiation_diffuse_horizontal,month_sin,month_cos,day_sin,day_cos,hour_sin,hour_cos,weekday_sin,weekday_cos,load_actual_pred,solar_generation_pred,wind_generation_pred,price_day_ahead_pred,temperature_pred,radiation_direct_horizontal_pred,radiation_diffuse_horizontal_pred
0,2015-01-01 00:00:00+00:00,0,-1.0,0,0,0,3,0.333333,-0.868881,-0.999995,-0.999917,-0.036019,-0.779024,-1.0,-1.0,0.0,1.000000,0.000000,1.00000,0.000000,1.000000,0.433884,-0.900969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2015-01-01 01:00:00+00:00,0,-1.0,0,0,1,3,0.333333,-0.867209,-0.999996,-0.999286,-0.052502,-0.770590,-1.0,-1.0,0.0,1.000000,0.000000,1.00000,0.258819,0.965926,0.433884,-0.900969,-1.161181,-1.097711,-1.124598,-0.212668,-0.786290,-1.410044,-0.916272
2,2015-01-01 02:00:00+00:00,0,-1.0,0,0,2,3,0.333333,-0.872605,-0.999995,-0.999347,-0.110902,-0.759535,-1.0,-1.0,0.0,1.000000,0.000000,1.00000,0.500000,0.866025,0.433884,-0.900969,-1.116041,-1.080096,-1.099129,-0.206513,-0.769359,-1.358665,-0.889505
3,2015-01-01 03:00:00+00:00,0,-1.0,0,0,3,3,0.333333,-0.873883,-0.999995,-0.999300,-0.139392,-0.747129,-1.0,-1.0,0.0,1.000000,0.000000,1.00000,0.707107,0.707107,0.433884,-0.900969,-1.041792,-1.061131,-1.008731,-0.207184,-0.751144,-1.299256,-0.858701
4,2015-01-01 04:00:00+00:00,0,-1.0,0,0,4,3,0.333333,-0.881053,-0.999995,-0.999484,-0.190042,-0.737589,-1.0,-1.0,0.0,1.000000,0.000000,1.00000,0.866025,0.500000,0.433884,-0.900969,-0.953795,-1.040070,-0.934987,-0.184219,-0.730938,-1.229738,-0.829269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175291,2019-12-31 19:00:00+00:00,261,3.0,11,30,19,1,1.000000,-0.167290,-1.000000,-0.754123,-0.100095,-0.185866,-1.0,-1.0,-0.5,0.866025,-0.201299,0.97953,-0.965926,0.258819,0.781831,0.623490,-0.083458,-1.070426,-0.493131,0.045321,-0.216586,-1.197622,-1.133252
175292,2019-12-31 20:00:00+00:00,261,3.0,11,30,20,1,1.000000,-0.225331,-1.000000,-0.742449,-0.114941,-0.191598,-1.0,-1.0,-0.5,0.866025,-0.201299,0.97953,-0.866025,0.500000,0.781831,0.623490,-0.205058,-1.085295,-0.497325,-0.043705,-0.224200,-1.257187,-1.148364
175293,2019-12-31 21:00:00+00:00,261,3.0,11,30,21,1,1.000000,-0.283632,-1.000000,-0.738013,-0.153037,-0.201384,-1.0,-1.0,-0.5,0.866025,-0.201299,0.97953,-0.707107,0.707107,0.781831,0.623490,-0.341897,-1.094167,-0.564585,-0.105669,-0.229758,-1.298907,-1.158824
175294,2019-12-31 22:00:00+00:00,261,3.0,11,30,22,1,1.000000,-0.361136,-1.000000,-0.716551,-0.169083,-0.213626,-1.0,-1.0,-0.5,0.866025,-0.201299,0.97953,-0.500000,0.866025,0.781831,0.623490,-0.452277,-1.098482,-0.595025,-0.166660,-0.237636,-1.326830,-1.156580


### -------------------- run RNN on energy data --------------------

#### train rnn baseline

In [17]:
training_samples=num_samples_train
test_samples=num_samples_test
timesteps=num_timesteps
num_atoms=num_ts
dims=num_dims
lstm_layers = 1
input_dim = num_atoms+len(cols_timedims)
target_dim = num_atoms
hidden_dim = 128

rnn_args = parse_args(
    epochs=epochs, 
    training_samples=training_samples, 
    test_samples=test_samples,
    shuffle_traindata=True,
    suffix=suffix,
    timesteps=timesteps,
    num_atoms=num_atoms,
    dims=dims,
    no_cuda=no_cuda,
    dont_use_encoder=dont_use_encoder,
    batch_size=batch_size)

In [18]:
if run_rnn:
    
    start_time = time.time()
    print(f"Starting_time: {datetime.datetime.fromtimestamp(start_time)}")
    
    logs = Logger(rnn_args)

    if rnn_args.GPU_to_use is not None:
        logs.write_to_log_file("Using GPU #" + str(rnn_args.GPU_to_use))

    (
        train_loader,
        valid_loader,
        test_loader,
        loc_max,
        loc_min,
        vel_max,
        vel_min,
    ) = load_data(rnn_args)
    
    rnn = RNN_baseline(lstm_layers, 
                       input_dim, 
                       target_dim, 
                       batch_size, 
                       hidden_dim,
                      rnn_args.device)
    if rnn_args.cuda:
        rnn = rnn.cuda()
    
    optimizer = optim.Adam(list(rnn.parameters()), lr=rnn_args.lr)
    scheduler = lr_scheduler.StepLR(optimizer, 
                                    step_size=rnn_args.lr_decay, 
                                    gamma=rnn_args.gamma)
    
    logs.write_to_log_file(rnn)

    ##Train model
    try:
        if rnn_args.test_time_adapt:
            raise KeyboardInterrupt

        best_epoch, epoch = train_rnn(rnn_args)

    except KeyboardInterrupt:
        best_epoch, epoch = -1, -1

    print("Optimization Finished!")
    logs.write_to_log_file("Best Epoch: {:04d}".format(best_epoch))

    if rnn_args.test:
        test_rnn(rnn, epoch, rnn_args)
    
    logs.write_to_log_file('Data: '+ suffix)
    logs.write_to_log_file('Version: '+ version)
    
    end_time = time.time()
    total_time = end_time-start_time
    print(f'Total Time: {time.strftime("%H:%M:%S", time.gmtime(total_time))}')
    logs.write_to_log_file(f'Total Time: {time.strftime("%H:%M:%S", time.gmtime(total_time))}')
    